In [1]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [11]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

docs = []
for doc in ["documents/signoff.docx", "documents/commentary.docx"]:
    loader = UnstructuredWordDocumentLoader(doc)
    docs.extend(loader.load())


In [12]:
docs


[Document(metadata={'source': 'documents/signoff.docx'}, page_content='To signoff on any task you have to do the following:\n\n1. Select the workpackage you want to work on from the left pane of the screen.\n\n2. If the workpackage is disabled or not assigned to you, click the Assign to me button located under the workpackage name.\n\n3. If the workpackage is assigned to someone else, please contact the person and inform them to unassign themselves or contact the Central Team if the person is unavailable.\n\n4. After clicking on the workpackage the list of signoffs will appear on the center on the page.\n\n5. Select the signoff you want to work on. A popup will appear.\n\n6. In the popup assert either yes or no on all the questions that appear. Select the signoff type from the dropdown located at the bottom of the page.\n\n7. If the signoff with caveat type is selected, you have to enter a caveat commentary in the text box that appears. The message must be between the range of 10 – 350

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("How to perform signoff?")

len(retrieved_docs)


Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


3

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

template = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""

human_prompt = HumanMessagePromptTemplate.from_template(template)
prompt = ChatPromptTemplate.from_messages([human_prompt])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [20]:
rag_chain.invoke("How to perform signoff?")


Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


"To perform a signoff, select the workpackage you want to work on and ensure it's assigned to you. Then, choose the signoff you wish to complete, answer the questions in the popup, and select the signoff type. Once all fields are filled, click the signoff button to save, changing the status from Pending to Complete."

In [21]:
rag_chain.invoke("I am trying to signoff but the signoff button is disabled, what to do?")


Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


'If the signoff button is disabled, ensure that all required fields are filled out correctly, including any caveat commentary if applicable. The message must be within the range of 10 to 3500 characters. Once all fields are completed, the signoff button should become enabled.'

In [22]:
rag_chain.invoke("I can't open my workpackage.")


Number of requested results 6 is greater than number of elements in index 3, updating n_results = 3


'To open your workpackage, ensure you have selected it from the left pane. If it\'s disabled or assigned to someone else, click "Assign to me" or contact the current assignee or Central Team. After selecting the workpackage, you should see the list of signoffs in the center of the page.'